## Testing CNN implentation with Keras

In [1]:
# Pathing Libraries
from pathlib import Path
import os
import os.path as op
import glob

# Image Libraries
import nibabel as nib
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras


In [8]:
## Pathing
path = '../'

images_path = Path(path, 'data', 'images')
masks_path = Path(path, 'data', 'masks')
segs_path = Path(path, 'data', 'segs_refs')
metadata_path = Path(path, 'data', 'meta')


In [13]:
meta_data = pd.read_csv(Path(metadata_path, 'meta_data_all.csv'))
#meta_data

for subj, age, gender in zip(meta_data['subject_id'], meta_data['age'], meta_data['gender_code']):
    print(subj, age, gender)

CC110033 24 1
CC110037 18 1
CC110045 24 2
CC110056 22 2
CC110062 20 1
CC110069 28 2
CC110087 28 2
CC110098 23 1
CC110101 23 1
CC110126 22 2
CC110174 25 2
CC110182 18 2
CC110187 25 2
CC110319 28 2
CC110411 25 1
CC110606 20 1
CC112141 29 1
CC120008 26 1
CC120049 28 1
CC120061 19 1
CC120065 25 2
CC120120 25 1
CC120123 19 2
CC120166 28 1
CC120182 26 1
CC120208 24 2
CC120218 27 2
CC120234 24 2
CC120264 28 1
CC120276 23 2
CC120286 22 1
CC120309 27 1
CC120313 28 1
CC120319 27 2
CC120347 21 2
CC120376 18 2
CC120409 18 1
CC120462 18 2
CC120469 25 2
CC120470 28 2
CC120550 19 1
CC120640 26 2
CC120727 23 2
CC120764 27 1
CC120795 24 1
CC120816 23 1
CC120987 20 2
CC121106 28 2
CC121111 18 1
CC121144 26 1
CC121158 28 2
CC121194 24 2
CC121200 26 2
CC121317 25 2
CC121397 27 1
CC121411 26 2
CC121428 26 2
CC121479 26 2
CC121685 20 1
CC121795 25 2
CC122172 27 1
CC122405 27 1
CC122620 24 2
CC210023 31 1
CC210051 35 2
CC210088 36 2
CC210124 35 2
CC210148 30 2
CC210172 31 2
CC210182 31 2
CC210250 29 1
CC2103

### Keras Implementation